In [3]:
from datasets import load_dataset

In [3]:
with open("done.txt","r") as f:
    print(f.read().splitlines())

['14499029']


In [5]:
pubmed_qa = load_dataset("qiaojin/PubMedQA","pqa_labeled")

Generating train split: 100%|██████████| 1000/1000 [00:00<00:00, 20828.22 examples/s]


In [7]:
pubmed_qa

DatasetDict({
    train: Dataset({
        features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
        num_rows: 1000
    })
})

In [9]:
pubmed_qa['train'][0]

{'pubid': 21645374,
 'question': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?',
 'context': {'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.',
   'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), ce

In [11]:
all_pubmed_ids = [x['pubid'] for x in pubmed_qa['train']]

In [40]:
# https://pubmed.ncbi.nlm.nih.gov/21645374/

from bs4 import BeautifulSoup
import requests

pubmed_id = "21645374"
base_url = "https://pubmed.ncbi.nlm.nih.gov/"


url = base_url + "21645374/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
doi_elem = soup.find(attrs={'class':'identifier doi'}).find(attrs={'class':'id-link'}).get("href")

doi_response = requests.get(doi_elem)
doi_soup = BeautifulSoup(doi_response.text, "html.parser")
ps = doi_soup.find_all("p")

In [41]:
for p in ps:
    print(len(p.get_text().strip().split(" ")))
    print(p.get_text())
    print("-"*100)

1
Advertisement
----------------------------------------------------------------------------------------------------
19

BMC Plant Biology
volume 11, Article number: 102 (2011)
            Cite this article

----------------------------------------------------------------------------------------------------
2
8778 Accesses
----------------------------------------------------------------------------------------------------
2
32 Citations
----------------------------------------------------------------------------------------------------
2
6 Altmetric
----------------------------------------------------------------------------------------------------
2
Metrics details
----------------------------------------------------------------------------------------------------
11
An Erratum to this article was published on 02 April 2013
----------------------------------------------------------------------------------------------------
83
Programmed cell death (PCD) is the regulated death of cells

In [23]:
doi_elem = soup.find(attrs={'class':'identifier doi'}).find(attrs={'class':'id-link'}).get("href")
# https://bmcplantbiol.biomedcentral.com/counter/pdf/10.1186/1471-2229-11-102.pdf
doi_elem

'https://doi.org/10.1186/1471-2229-11-102'

In [24]:
soup.find(attrs={'class':'identifier doi'}).find(attrs={'class':'id-link'})

<a class="id-link" data-ga-action="DOI" data-ga-category="full_text" href="https://doi.org/10.1186/1471-2229-11-102" ref="linksrc=article_id_link&amp;article_id=10.1186/1471-2229-11-102&amp;id_type=DOI" rel="noopener" target="_blank">
      10.1186/1471-2229-11-102
    </a>

In [ ]:
response = requests.get(doi_elem)
soup_doi = BeautifulSoup(response.text, "html.parser")


In [55]:
df_dict = {
    "pid":[],
    "url":[],
    "paragraph":[],
    "doi":[]
}

done_list = []

In [64]:
all_pubmed_ids[2]

9488747

In [63]:
import pandas as pd
from tqdm import tqdm

base_url = "https://pubmed.ncbi.nlm.nih.gov/"


idx = 0
for pid in tqdm(all_pubmed_ids):
    url = base_url + str(pid) + "/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    with open("done.txt","r") as f:
        done_list = f.read().split("\n")
    if str(pid) in done_list: 
        print(f'Results for {pid} are already done!')
        continue
    try:
        doi_elem = soup.find(attrs={'class':'identifier doi'})
        link = doi_elem.find(attrs={'class':'id-link'})
        doi_href = link.get("href")

        doi_response = requests.get(doi_href)
        doi_soup = BeautifulSoup(doi_response.text, "html.parser")
        doi_ps = doi_soup.find_all("p")
        paragraphs_list = []
        for p in doi_ps:
            para_len = len(p.get_text().strip().split(" "))
            if para_len > 20:
                para_text = p.get_text()
                paragraphs_list.append(para_text)
        df_dict["pid"].append(pid)
        df_dict["url"].append(url)
        paragraphs = " ".join(paragraphs_list)
        df_dict["paragraph"].append(paragraphs)
        df_dict["doi"].append("Yes")
        done_list.append(pid)
        df = pd.DataFrame(df_dict)
        df.to_csv("pubmed.csv", index=False)
        with open('done.txt', 'a') as f:
            # for line in lines:
            f.write(f"{pid}\n")
    except Exception as e:
        print(e)
        paras = soup.find_all("p")
        paragraphs_list = []
        for p in paras:
            para_len = len(p.get_text().strip().split(" "))
            if para_len > 20:
                para_text = p.get_text()
                paragraphs_list.append(para_text)
        paragraphs = " ".join(paragraphs)
        df_dict["pid"].append(pid)
        df_dict["url"].append(url)
        paragraphs = " ".join(paragraphs_list)
        df_dict["paragraph"].append(paragraphs)
        df_dict["doi"].append("No")
        done_list.append(pid)
        df = pd.DataFrame(df_dict)
        df.to_csv("pubmed.csv", index=False)
        with open('done.txt', 'a') as f:
            # for line in lines:
            f.write(f"{pid}\n")
    if idx == 2: break
    idx += 1
    # doi_response = requests.get(doi_elem)
    # soup_doi = BeautifulSoup(doi_response.text, "html.parser")

  0%|          | 2/1000 [03:45<31:13:59, 112.66s/it]


In [53]:
all_pubmed_ids[:4]

[21645374, 16418930, 9488747, 17208539]

In [51]:
df_dict

{'pid': [21645374],
 'url': ['https://pubmed.ncbi.nlm.nih.gov/21645374/'],
 'paragraph': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (L

In [49]:
doi_href

'https://doi.org/10.1186/1471-2229-11-102'

In [61]:
df_dict['paragraph'][2]

''

In [62]:
df_dict['url'][2]

'https://pubmed.ncbi.nlm.nih.gov/9488747/'

In [69]:
from langchain_community.utilities.pubmed import PubMedAPIWrapper

tool = PubMedAPIWrapper()
s = tool.run(query = "(21645374[UID])")

In [70]:
print(s)

Published: 2011-06-06
Title: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?
Copyright Information: 
Summary::
BACKGROUND: Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.
RESULTS: The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo

In [85]:
df_dict = {
    "pid":[],
    "url":[],
    "abstract":[]
}

done_idxs = []

In [95]:
from tqdm import tqdm
import time
tool = PubMedAPIWrapper()
pbar = tqdm(total=len(all_pubmed_ids),desc="Pubmed Summary")
for idx,pid in enumerate(all_pubmed_ids):
    if idx in done_idxs: 
        # print(f"Already done for {pid}")
        pbar.update(1)
        continue
    url = f"https://pubmed.ncbi.nlm.nih.gov/{pid}/"
    s = tool.load(query = f"({pid}[UID])")
    df_dict['url'].append(url)
    df_dict['pid'].append(pid)
    df_dict['abstract'].append(s[0]['Summary'])
    done_idxs.append(idx)
    time.sleep(1)
    pbar.update(1)

Pubmed Summary: 100%|██████████| 1000/1000 [20:17<00:00,  1.72s/it]

Pubmed Summary:  25%|██▌       | 252/1000 [00:18<00:04, 161.10it/s]

In [94]:
len(df_dict['abstract'])

241

In [96]:
import pandas as pd

df = pd.DataFrame(df_dict)
df.to_csv("pubmed_abstract.csv", index=False)